# Data Processing & Cleaning

In [10]:
import pandas as pd 
df1 = pd.read_csv("Data_sets/companies_client.csv")
df2 = pd.read_csv("Data_sets/companies_full_corrected.csv")


In [11]:
df1.head(10)

,CLIENT
0,Adobe Systems
1,Adventist Health
2,AECOM
3,Aerojet Rocketdyne
4,Alameda Transit
5,Alaska Community Foundation
6,Alaska Retirement Board
7,Alexander & Baldwin
8,Allergan
9,Alphabet


In [12]:
df2.head(10)

,FIRM_NAME
0,Adobe Systems Incorporated
1,Adventist Health System
2,AECOM Technology Corporation
3,Aerojet Rocketdyne Holdings Inc.
4,Alameda-Contra Costa Transit District
5,Alaska Community Foundation
6,Alaska Retirement Management Board
7,"Alexander & Baldwin, Inc."
8,"Allergan, Inc."
9,Alphabet Inc.


#### Itertools.product in Entity Resolution and Fuzzy Matching



**Primary Use**: `itertools.product` is primarily used for Entity Resolution or Fuzzy Matching. It generates a Cartesian product of two lists, pairing every element from List A with every element from List B.



**Application Scenario**:
- You have a list of "clean" company names (Reference Data) and a list of "dirty" company names (Input Data).
- To find the best match, compare every incoming name against every reference name.
- `itertools.product(list_a, list_b)` handles the logic of creating these pairs, allowing you to calculate similarity scores (e.g., Levenshtein distance) for each pair.



![alt text](itertools-product.webp)



### Create a Dataframe That Conatains All Possible Combinations of Two Columns From Two Different Dataframes Using Itertools.product.

In [13]:
import itertools
df = pd.DataFrame(itertools.product(df1["CLIENT"],df2["FIRM_NAME"]),columns=["CSV 1","CSV 2"])
df.head(10)

,CSV 1,CSV 2
0,Adobe Systems,Adobe Systems Incorporated
1,Adobe Systems,Adventist Health System
2,Adobe Systems,AECOM Technology Corporation
3,Adobe Systems,Aerojet Rocketdyne Holdings Inc.
4,Adobe Systems,Alameda-Contra Costa Transit District
5,Adobe Systems,Alaska Community Foundation
6,Adobe Systems,Alaska Retirement Management Board
7,Adobe Systems,"Alexander & Baldwin, Inc."
8,Adobe Systems,"Allergan, Inc."
9,Adobe Systems,Alphabet Inc.


In [14]:
df.shape

(2704, 2)

## Levenshtein Distance
- **Definition**: Metric for measuring the difference between two sequences (usually strings). Represents the minimum number of single-character edits required to change one word into the other.
- **Allowed Operations**:
  - Insertion (adding a character)
  - Deletion (removing a character)
  - Substitution (replacing a character)



## Fuzz.ratio
- **Purpose**: Calculates similarity between two strings as a score from 0 to 100. Essentially a normalized Levenshtein distance.
- **How it Works**: Compares the entire first string to the entire second string.
- **Formula (Conceptual)**: Roughly $2 \times M / T$, where $M$ is the number of matches and $T$ is the total characters in both strings.
- **Behavior**: Strict on length differences. E.g., "this is a test" vs. "this is a test!" drops slightly; significant length diffs drop score heavily.
- **When to Use**: For nearly identical strings in length and content (e.g., typo corrections like "Apple Inc." vs. "Aple Inc.").



## Fuzz.partial_ratio
- **Purpose**: Handles substring matching, addressing low scores from length differences in fuzz.ratio.
- **How it Works**:
  - Takes the shorter string (length $L$).
  - Scans the longer string with a sliding window of length $L$.
  - Calculates ratio for each substring and returns the highest score.
- **Behavior**: E.g., "New York" and "New York City" score 100, as "New York" is a perfect substring. fuzz.ratio would score ~70-80.

In [15]:
from thefuzz import fuzz
from thefuzz import process

score= [fuzz.partial_ratio(c1,c2) for c1,c2 in df.values]
df["Ratio Score"] = score
df.head()

,CSV 1,CSV 2,Ratio Score
0,Adobe Systems,Adobe Systems Incorporated,100
1,Adobe Systems,Adventist Health System,70
2,Adobe Systems,AECOM Technology Corporation,29
3,Adobe Systems,Aerojet Rocketdyne Holdings Inc.,40
4,Adobe Systems,Alameda-Contra Costa Transit District,32


In [16]:
df.loc[df["Ratio Score"] <=90]

,CSV 1,CSV 2,Ratio Score
1,Adobe Systems,Adventist Health System,70
2,Adobe Systems,AECOM Technology Corporation,29
3,Adobe Systems,Aerojet Rocketdyne Holdings Inc.,40
4,Adobe Systems,Alameda-Contra Costa Transit District,32
5,Adobe Systems,Alaska Community Foundation,26
...,...,...,...
2698,Walmart,"Salesforce, Inc.",43
2699,Walmart,Starbucks Corporation,36
2700,Walmart,"Tesla, Inc.",33
2701,Walmart,"Uber Technologies, Inc.",18


In [17]:
df.loc[(df["CSV 1"]=="Apple") & (df["Ratio Score"]>=80)]

,CSV 1,CSV 2,Ratio Score
689,Apple,Apple Inc.,100


In [19]:
df.loc[(df["CSV 1"]=="Alameda Transit") & (df["Ratio Score"]>50)]

,CSV 1,CSV 2,Ratio Score
212,Alameda Transit,Alameda-Contra Costa Transit District,67
213,Alameda Transit,Alaska Community Foundation,53
219,Alameda Transit,American Airlines Group Inc.,54
220,Alameda Transit,American Express Company,52
